# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [107]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Ask for parameters

In [125]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [126]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Sales:**

*  **S:** Positive deviation from the sales target of 5500 units.
*  **s:** Negative deviation from the sales target of 5500 units. 

**Employee Hours:**

*  **F:** Positive deviation from the target of 800 hours for full-time employees.
*  **f:** Negative deviation from the target of 800 hours for full-time employees.
*  **P:** Positive deviation from the target of 320 hours for part-time employees.
*  **p:** Negative deviation from the target of 320 hours for part-time employees.

**Overtime:**

*  **O:** Positive deviation from the target of 100 hours of overtime for full-time employees.
*  **o:** Negative deviation from the target of 100 hours of overtime for full-time employees.
*  **E:** Total overtime hours for part-time employees. 

**Priority Factors:**

* **P1:** Priority factor for deviations from the sales target (S and s).
* **P2:** Priority factor for exceeding overtime target for full-time employees (O).
* **P3:** Priority factor for fully utilizing full-time and part-time employee hours (F, f, P, p).
* **P4:** Priority factor for minimizing overall overtime (O, o, E). 


# 2. Ask for objective

In [127]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [128]:
Markdown(response2.text)

```
Minimize: 

P1 * (S + s) + P2 * O + P3 * (2*F + 2*f + P + p) + P4 * (O + o + 3*E) 
``` 


# 3. Ask for constraints

In [129]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [130]:
Markdown(response3.text)

## Constraints:

**Sales:**

*  5 * (800 + F - f) + 2 * (320 + P - p) + 5 * (100 + O - o) + 2 * E = 5500 + S - s 
   *(This equation connects sales with employee hours and overtime. It ensures the total sales, considering the working hours and productivity of full-time and part-time employees, match the target sales plus any deviations)*

**Non-negativity:**

*  S, s, F, f, P, p, O, o, E ≥ 0 
   *(All deviation variables must be non-negative)* 


## 3. Generate the pyomo code

In [131]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [132]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
P1 = 1  
P2 = 1.2
P3 = 0.8
P4 = 1.5

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.s = pyo.Var(within=pyo.NonNegativeReals)
model.F = pyo.Var(within=pyo.NonNegativeReals)
model.f = pyo.Var(within=pyo.NonNegativeReals)
model.P = pyo.Var(within=pyo.NonNegativeReals)
model.p = pyo.Var(within=pyo.NonNegativeReals)
model.O = pyo.Var(within=pyo.NonNegativeReals)
model.o = pyo.Var(within=pyo.NonNegativeReals)
model.E = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr = 
                           P1 * (model.S + model.s) + 
                           P2 * model.O + 
                           P3 * (2*model.F + 2*model.f + model.P + model.p) + 
                           P4 * (model.O + model.o + 3*model.E),
                           sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Constraint(expr = 
                                        5 * (800 + model.F - model.f) + 
                                        2 * (320 + model.P - model.p) + 
                                        5 * (100 + model.O - model.o) + 
                                        2 * model.E == 5500 + model.S - model.s)

# Solve
solver = pyo.SolverFactory('cbc')  # Choose a solver (e.g., 'glpk', 'cbc')
solver.solve(model)

# Display Results
print("Solution:")
print(f"Sales Deviation (Positive): S = {pyo.value(model.S)}")
print(f"Sales Deviation (Negative): s = {pyo.value(model.s)}")
print(f"Full-Time Hours Deviation (Positive): F = {pyo.value(model.F)}")
print(f"Full-Time Hours Deviation (Negative): f = {pyo.value(model.f)}")
print(f"Part-Time Hours Deviation (Positive): P = {pyo.value(model.P)}")
print(f"Part-Time Hours Deviation (Negative): p = {pyo.value(model.p)}")
print(f"Full-Time Overtime Deviation (Positive): O = {pyo.value(model.O)}")
print(f"Full-Time Overtime Deviation (Negative): o = {pyo.value(model.o)}")
print(f"Part-Time Overtime: E = {pyo.value(model.E)}")
print(f"Objective Value: {pyo.value(model.obj)}") 
```

In this code:

1.  **Sample Data:** The values for `P1`, `P2`, `P3`, and `P4` are placeholders. You should replace these with your actual priority factor values.
2.  **Model:** We create a concrete model using `pyo.ConcreteModel()`.
3.  **Variables:** We define all the decision variables (`S`, `s`, `F`, `f`, `P`, `p`, `O`, `o`, `E`) using `pyo.Var()` and specify that they are non-negative.
4.  **Objective Function:** We define the objective function to minimize, incorporating the priority factors and the deviations.
5.  **Constraints:** We define the constraint connecting sales, employee hours, and overtime.
6.  **Solver:** We choose a solver (here, 'cbc') using `pyo.SolverFactory()` and solve the model.
7.  **Results:** We display the optimal values for each decision variable and the objective function value.


## 4. Input problem data and try running the generated code

In [1]:
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
P1 = 4
P2 = 3
P3 = 2
P4 = 1

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.s = pyo.Var(within=pyo.NonNegativeReals)
model.F = pyo.Var(within=pyo.NonNegativeReals)
model.f = pyo.Var(within=pyo.NonNegativeReals)
model.P = pyo.Var(within=pyo.NonNegativeReals)
model.p = pyo.Var(within=pyo.NonNegativeReals)
model.O = pyo.Var(within=pyo.NonNegativeReals)
model.o = pyo.Var(within=pyo.NonNegativeReals)
model.E = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr = 
                           P1 * (model.S + model.s) + 
                           P2 * model.O + 
                           P3 * (2*model.F + 2*model.f + model.P + model.p) + 
                           P4 * (model.O + model.o + 3*model.E),
                           sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Constraint(expr = 
                                        5 * (800 + model.F - model.f) + 
                                        2 * (320 + model.P - model.p) + 
                                        5 * (100 + model.O - model.o) + 
                                        2 * model.E == 5500 + model.S - model.s)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a solver (e.g., 'glpk', 'cbc')
solver.solve(model)

# Display Results
print("Solution:")
print(f"Sales Deviation (Positive): S = {pyo.value(model.S)}")
print(f"Sales Deviation (Negative): s = {pyo.value(model.s)}")
print(f"Full-Time Hours Deviation (Positive): F = {pyo.value(model.F)}")
print(f"Full-Time Hours Deviation (Negative): f = {pyo.value(model.f)}")
print(f"Part-Time Hours Deviation (Positive): P = {pyo.value(model.P)}")
print(f"Part-Time Hours Deviation (Negative): p = {pyo.value(model.p)}")
print(f"Full-Time Overtime Deviation (Positive): O = {pyo.value(model.O)}")
print(f"Full-Time Overtime Deviation (Negative): o = {pyo.value(model.o)}")
print(f"Part-Time Overtime: E = {pyo.value(model.E)}")
print(f"Objective Value: {pyo.value(model.obj)}")

Solution:
Sales Deviation (Positive): S = 0.0
Sales Deviation (Negative): s = 0.0
Full-Time Hours Deviation (Positive): F = 0.0
Full-Time Hours Deviation (Negative): f = 0.0
Part-Time Hours Deviation (Positive): P = 0.0
Part-Time Hours Deviation (Negative): p = 0.0
Full-Time Overtime Deviation (Positive): O = 72.0
Full-Time Overtime Deviation (Negative): o = 0.0
Part-Time Overtime: E = 0.0
Objective Value: 288.0


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [135]:
print(response.text)

## Variables:

**Sales:**

*  **S:** Positive deviation from the sales target of 5500 units.
*  **s:** Negative deviation from the sales target of 5500 units. 

**Employee Hours:**

*  **F:** Positive deviation from the target of 800 hours for full-time employees.
*  **f:** Negative deviation from the target of 800 hours for full-time employees.
*  **P:** Positive deviation from the target of 320 hours for part-time employees.
*  **p:** Negative deviation from the target of 320 hours for part-time employees.

**Overtime:**

*  **O:** Positive deviation from the target of 100 hours of overtime for full-time employees.
*  **o:** Negative deviation from the target of 100 hours of overtime for full-time employees.
*  **E:** Total overtime hours for part-time employees. 

**Priority Factors:**

* **P1:** Priority factor for deviations from the sales target (S and s).
* **P2:** Priority factor for exceeding overtime target for full-time employees (O).
* **P3:** Priority factor for fully util

In [136]:
print(response2.text)

```
Minimize: 

P1 * (S + s) + P2 * O + P3 * (2*F + 2*f + P + p) + P4 * (O + o + 3*E) 
``` 



In [137]:
print(response3.text)

## Constraints:

**Sales:**

*  5 * (800 + F - f) + 2 * (320 + P - p) + 5 * (100 + O - o) + 2 * E = 5500 + S - s 
   *(This equation connects sales with employee hours and overtime. It ensures the total sales, considering the working hours and productivity of full-time and part-time employees, match the target sales plus any deviations)*

**Non-negativity:**

*  S, s, F, f, P, p, O, o, E ≥ 0 
   *(All deviation variables must be non-negative)* 



In [138]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
P1 = 1  
P2 = 1.2
P3 = 0.8
P4 = 1.5

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.s = pyo.Var(within=pyo.NonNegativeReals)
model.F = pyo.Var(within=pyo.NonNegativeReals)
model.f = pyo.Var(within=pyo.NonNegativeReals)
model.P = pyo.Var(within=pyo.NonNegativeReals)
model.p = pyo.Var(within=pyo.NonNegativeReals)
model.O = pyo.Var(within=pyo.NonNegativeReals)
model.o = pyo.Var(within=pyo.NonNegativeReals)
model.E = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(expr = 
                           P1 * (model.S + model.s) + 
                           P2 * model.O + 
                           P3 * (2*model.F + 2*model.f + model.P + model.p) + 
                           P4 * (model.O + model.o + 3*model.E),
                           sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Con